In [ ]:
import utils
import numpy as np
import glob
import os
import pandas as pd

print(np.__version__)
from first_surface_parser import extract_and_save
import ray

Get Available Categories

In [ ]:
def available_categories(ims_file_path: str, valid_surface: int):
    # load the imaris file
    data = utils.load_ims(ims_file_path)

    # get surface we want to parse
    surface_name = utils.get_object_names(full_data_file=data, search_for="Surface")[
        valid_surface
    ]

    # get all the statistics names
    surface_stats_names = utils.get_statistics_names(
        full_data_file=data, object_name=surface_name
    )

    # get the statistics values in the surface
    surface_stats_values = utils.get_stats_values(
        full_data_file=data, object_name=surface_name
    )

    # get only the traack ids
    track_ids = utils.get_first_timeframe_objects(
        stats_values=surface_stats_values, stats_dict=surface_stats_names
    )

    # filter full stats values based on track ids
    filtered_stats_df = utils.filter_by_object_id(
        stats_values=surface_stats_values, object_ids=track_ids
    )

    # filter stats dict
    filtered_dict = utils.filter_stats_dict(
        stats_dict=surface_stats_names, filtered_df=filtered_stats_df
    )

    # invert dictionary + name modifications
    # this step is a cosmetic step
    inverted_stats_names = utils.invert_stats_dict(filtered_dict)
    inverted_stats_names = utils.flatten(inverted_stats_names)

    # reverse the stats names again such that key=num, value=name
    # final_stats_names = {v: k for k,v in inverted_stats_names.items()}

    np.savetxt("stats_categories_new3.txt", list(inverted_stats_names.keys()), fmt="%s")

    print(
        f"Saved Stats Categories in directory: {os.path.dirname(os.path.abspath(ims_file_path))}"
    )

    # return list(inverted_stats_names.keys())

In [ ]:
def generate_available_categories(config_path: str):
    # load config path
    yaml = utils.load_yaml("config.yaml")

    # files to scan
    directories = yaml["data_dir"]

    # valid surface
    valid_surface = int(yaml["valid_surface"]) - 1

    for directory in directories:
        # grab all the files in the directory w/ .ims
        filenames = list(glob.glob(os.path.join(directory, "*.ims")))

        for filename in filenames:
            # file path
            file_path = filename

            # print(file_path, filename, 'dafd')

            # get and save the available categories csv file
            available_categories(file_path, valid_surface)

            break

        break

    print("[info] Please Edit The Statistics File")

In [ ]:
# RUN GENERATE CATEGORIES
# generate_available_categories("config.yaml")

Generate Statistics 

In [ ]:
def generate_statistics(config_path: str):
    # load config path
    yaml = utils.load_yaml("config.yaml")

    # files to scan
    directories = yaml["data_dir"]

    # get the stats categories
    stats_categories = utils.read_txt(yaml["stats_category_path"])

    # valid surface
    valid_surface = int(yaml["valid_surface"]) - 1

    # create a list to hold ray subprocess
    processes = []

    for directory in directories:
        # grab all the files in the directory w/ .ims
        filenames = list(glob.glob(os.path.join(directory, "*.ims")))

        for filename in filenames:
            # file path
            file_path = filename  # os.path.join(directory, filename)

            # save_file_path
            save_path = os.path.splitext(filename)[0] + ".csv"
            # save_path = os.path.join(directory, save_path)

            # extract and save
            processes.append(
                extract_and_save.remote(
                    file_path, valid_surface, stats_categories, save_path
                )
            )

    print("info -- running subprocesses:")

    ray.get(processes)

In [ ]:
# RUN GENERATE STATISTICS
generate_statistics(config_path="config.yaml")

In [ ]:
# Working